In [2]:
# smile 뜨게 만드는 작업 했어욯ㅎㅎ
import cv2
import sys
import numpy as np

faceCascade = cv2.CascadeClassifier('D:/javaForever/util/opencv/sources/data/haarcascades/haarcascade_frontalface_alt.xml')
eyeCascade = cv2.CascadeClassifier('D:/javaForever/util/opencv/sources/data/haarcascades/haarcascade_eye.xml')
mouthCascade = cv2.CascadeClassifier('D:/javaForever/util/opencv/sources/data/haarcascades/haarcascade_mcs_mouth.xml')

def Rotate(src, degrees):
    if degrees == 90:
        dst = cv2.transpose(src) # 행렬 변경 
        dst = cv2.flip(dst, 1)   # 뒤집기

    elif degrees == 180:
        dst = cv2.flip(src, 0)   # 뒤집기

    elif degrees == 270:
        dst = cv2.transpose(src) # 행렬 변경
        dst = cv2.flip(dst, 0)   # 뒤집기
    else:
        dst = null
    return dst


video_capture = cv2.VideoCapture('HUN.mp4')
scaling_factor = 0.5

def findMouth(frame, eyes, faces):
    mouths = mouthCascade.detectMultiScale(
            frame,
            scaleFactor=1.2,
            minNeighbors=8,
            minSize=(30, 10),
            flags = cv2.CASCADE_SCALE_IMAGE)

    if len(mouths) <= 0:
        return 0, 0, 0, 0

    eyeBottom = 0

    for eye in eyes:
        eyeX, eyeY, eyeW, eyeH = eye
        if (eyeY + eyeH) > eyeBottom:
            eyeBottom = eyeY + eyeH

    faceX, faceY, faceW, faceH = faces[0]
    faceBottom = faceY + faceH

    for mouth in mouths:
        mouthX, mouthY, mouthW, mouthH = mouth
        if mouthY > (eyeBottom + 50) and (mouthY + mouthH) < faceBottom:
            return mouth

    return 0, 0, 0, 0

if __name__ == '__main__':
    while True:
        # Capture frame-by-frame
        ret, frame = video_capture.read()
        frame = Rotate(frame,90)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        gray2 = cv2.equalizeHist(gray)

        faces = faceCascade.detectMultiScale(
            gray,
            scaleFactor=1.2,
            minNeighbors=6,
            minSize=(50, 50),
            flags = cv2.CASCADE_SCALE_IMAGE)

        eyes = eyeCascade.detectMultiScale(
            gray,
            scaleFactor=1.2,
            minNeighbors=6,
            minSize=(50, 50),
            flags = cv2.CASCADE_SCALE_IMAGE)


        # Draw a rectangle around the faces
        for (x, y, w, h) in faces:
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
            
        # Draw a rectangle around the eyes
        if len(eyes) >= 2:
            eyes = eyes[:2]
        for (x, y, w, h) in eyes:
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 2)
        
        # Draw a rectangle around the mouth
        if len(faces) > 0 and len(eyes) > 0:
            x, y, w, h = findMouth(gray2, eyes, faces)
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
            cv2.putText(frame,'Smile',(x,y-7), 3, 1.2, (0, 255, 0), 2, cv2.LINE_AA)
            
        # Display the resulting frame
        cv2.imshow('Video', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # When everything is done, release the capture
video_capture.release()
cv2.destroyAllWindows()

error: OpenCV(3.4.1) C:\bld\opencv_1520732670222\work\opencv-3.4.1\modules\imgproc\src\color.cpp:11147: error: (-215) scn == 3 || scn == 4 in function cv::cvtColor


In [1]:
# 손인식인데 되지 않아.. 실행이 되지 않아ㅠㅠ
# 근데...어깨가 인식이 되넹ㅎㅎㅎ
import cv2
import numpy as np
import math

def Rotate(src, degrees):
    if degrees == 90:
        dst = cv2.transpose(src) # 행렬 변경 
        dst = cv2.flip(dst, 1)   # 뒤집기

    elif degrees == 180:
        dst = cv2.flip(src, 0)   # 뒤집기

    elif degrees == 270:
        dst = cv2.transpose(src) # 행렬 변경
        dst = cv2.flip(dst, 0)   # 뒤집기
    else:
        dst = null
    return dst
cap = cv2.VideoCapture('HUN.mp4')

while(cap.isOpened()):
    ret, img = cap.read()
    img = Rotate(img,90)
    cv2.rectangle(img,(400,400),(200,200),(0,255,0),0)
    crop_img = img[300:500, 150:360]
    grey = cv2.cvtColor(crop_img, cv2.COLOR_BGR2GRAY)
    value = (35, 35)
    blurred = cv2.GaussianBlur(grey, value, 0)
    _, thresh1 = cv2.threshold(blurred, 127, 255,
                               cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
    cv2.imshow('Thresholded', thresh1)
    _,contours, hierarchy = cv2.findContours(thresh1.copy(),cv2.RETR_TREE, \
            cv2.CHAIN_APPROX_NONE)
    max_area = -1
    for i in range(len(contours)):
        cnt=contours[i]
        area = cv2.contourArea(cnt)
        if(area>max_area):
            max_area=area
            ci=i
    cnt=contours[ci]
    x,y,w,h = cv2.boundingRect(cnt)
    cv2.rectangle(crop_img,(x,y),(x+w,y+h),(0,0,255),0)
    hull = cv2.convexHull(cnt)
    drawing = np.zeros(crop_img.shape,np.uint8)
    cv2.drawContours(drawing,[cnt],0,(0,255,0),0)
    cv2.drawContours(drawing,[hull],0,(0,0,255),0)
    hull = cv2.convexHull(cnt,returnPoints = False)
    defects = cv2.convexityDefects(cnt,hull)
    count_defects = 0
    cv2.drawContours(thresh1, contours, -1, (0,255,0), 3)
    for i in range(defects.shape[0]):
        s,e,f,d = defects[i,0]
        start = tuple(cnt[s][0])
        end = tuple(cnt[e][0])
        far = tuple(cnt[f][0])
        a = math.sqrt((end[0] - start[0])**2 + (end[1] - start[1])**2)
        b = math.sqrt((far[0] - start[0])**2 + (far[1] - start[1])**2)
        c = math.sqrt((end[0] - far[0])**2 + (end[1] - far[1])**2)
        angle = math.acos((b**2 + c**2 - a**2)/(2*b*c)) * 57
        if angle <= 90:
            count_defects += 1
            cv2.circle(crop_img,far,1,[0,0,255],-1)
        #dist = cv2.pointPolygonTest(cnt,far,True)
        cv2.line(crop_img,start,end,[0,255,0],2)
        #cv2.circle(crop_img,far,5,[0,0,255],-1)
    if count_defects == 1:
        cv2.putText(img,"I'M VIKRANT SHARMA", (50,50), cv2.FONT_HERSHEY_SIMPLEX, 2, 2)
    elif count_defects == 2:
        str = "THIS IS A BASIC HAND GESTURE RECOGNISER!!"
        cv2.putText(img, str, (5,50), cv2.FONT_HERSHEY_SIMPLEX, 1, 2)
    elif count_defects == 3:
        cv2.putText(img,"This is FOUR (:", (50,50), cv2.FONT_HERSHEY_SIMPLEX, 2, 2)
    elif count_defects == 4:
        cv2.putText(img,"HARE KRSNA", (50,50), cv2.FONT_HERSHEY_SIMPLEX, 2, 2)
    else:
        cv2.putText(img,"GOOD AFTERNOON TEACHERS", (50,50),\
                    cv2.FONT_HERSHEY_SIMPLEX, 2, 2)
    #cv2.imshow('drawing', drawing)
    #cv2.imshow('end', crop_img)
    cv2.imshow('Gesture', img)
    all_img = np.hstack((drawing, crop_img))
    cv2.imshow('Contours', all_img)
    k = cv2.waitKey(1)
    if k == 27:
        break
        
VideoCapture.release()
cv2.destroyAllWindows()

TypeError: 'NoneType' object is not subscriptable

In [13]:
#이미지에서 다리인식하기

import numpy as np
import cv2

img = cv2.imread('people.jpg',0)

upperBody_cascade = cv2.CascadeClassifier('D:/javaForever/util/opencv/sources/data/haarcascades/haarcascade_lowerbody.xml')    

arrUpperBody = upperBody_cascade.detectMultiScale(img)
if arrUpperBody != ():
        for (x,y,w,h) in arrUpperBody:
            cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)

cv2.imshow('image',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: elementwise != comparison failed; this will raise an error in the future.
  if __name__ == '__main__':


TypeError: image is not a numpy array, neither a scalar

In [8]:
# 손인식인데 되지 않아.. 실행이 되지 않아ㅠㅠ

import numpy as np
import cv2
import math

capture = cv2.VideoCapture('HUN.mp4')

while():
    
    # Capture frames from the camera
    ret, frame = capture.read()
    
    # Get hand data from the rectangle sub window   
    cv2.rectangle(frame,(100,100),(300,300),(0,255,0),0)
    crop_image = frame[100:300, 100:300]
    
    # Apply Gaussian blur
    blur = cv2.GaussianBlur(crop_image, (3,3), 0)
    
    # Change color-space from BGR -> HSV
    hsv = cv2.cvtColor(blur, cv2.COLOR_BGR2HSV)
    
    # Create a binary image with where white will be skin colors and rest is black
    mask2 = cv2.inRange(hsv, np.array([2,0,0]), np.array([20,255,255]))
    
    # Kernel for morphological transformation    
    kernel = np.ones((5,5))
    
    # Apply morphological transformations to filter out the background noise
    dilation = cv2.dilate(mask2, kernel, iterations = 1)
    erosion = cv2.erode(dilation, kernel, iterations = 1)    
       
    # Apply Gaussian Blur and Threshold
    filtered = cv2.GaussianBlur(erosion, (3,3), 0)
    ret,thresh = cv2.threshold(filtered, 127, 255, 0)
    
    # Show threshold image
    cv2.imshow("Thresholded", thresh)

    # Find contours
    image, contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE )
    
    
    try:
        # Find contour with maximum area
        contour = max(contours, key = lambda x: cv2.contourArea(x))
        
        # Create bounding rectangle around the contour
        x,y,w,h = cv2.boundingRect(contour)
        cv2.rectangle(crop_image,(x,y),(x+w,y+h),(0,0,255),0)
        
        # Find convex hull
        hull = cv2.convexHull(contour)
        
        # Draw contour
        drawing = np.zeros(crop_image.shape,np.uint8)
        cv2.drawContours(drawing,[contour],-1,(0,255,0),0)
        cv2.drawContours(drawing,[hull],-1,(0,0,255),0)
        
        # Find convexity defects
        hull = cv2.convexHull(contour, returnPoints=False)
        defects = cv2.convexityDefects(contour,hull)
        
        # Use cosine rule to find angle of the far point from the start and end point i.e. the convex points (the finger 
        # tips) for all defects
        count_defects = 0
        
        for i in range(defects.shape[0]):
            s,e,f,d = defects[i,0]
            start = tuple(contour[s][0])
            end = tuple(contour[e][0])
            far = tuple(contour[f][0])

            a = math.sqrt((end[0] - start[0])**2 + (end[1] - start[1])**2)
            b = math.sqrt((far[0] - start[0])**2 + (far[1] - start[1])**2)
            c = math.sqrt((end[0] - far[0])**2 + (end[1] - far[1])**2)
            angle = (math.acos((b**2 + c**2 - a**2)/(2*b*c))*180)/3.14
            
            # if angle > 90 draw a circle at the far point
            if angle <= 90:
                count_defects += 1
                cv2.circle(crop_image,far,1,[0,0,255],-1)

            cv2.line(crop_image,start,end,[0,255,0],2)

        # Print number of fingers
        if count_defects == 0:
            cv2.putText(frame,"HELLO", (50,50), cv2.FONT_HERSHEY_SIMPLEX, 2, 2)
        elif count_defects == 1:
            cv2.putText(frame,"TWO", (50,50), cv2.FONT_HERSHEY_SIMPLEX, 2, 2)
        elif count_defects == 2:
            cv2.putText(frame, "THREE", (5,50), cv2.FONT_HERSHEY_SIMPLEX, 2, 2)
        elif count_defects == 3:
            cv2.putText(frame,"FOUR", (50,50), cv2.FONT_HERSHEY_SIMPLEX, 2, 2)
        elif count_defects == 4:
            cv2.putText(frame,"FIVE", (50,50), cv2.FONT_HERSHEY_SIMPLEX, 2, 2)
        else:
            pass
    except:
        pass

    
    # Show required images
    cv2.imshow("Gesture", frame)
    all_image = np.hstack((drawing, crop_image))
    cv2.imshow('Contours', all_image)
      
    # Close the camera if 'q' is pressed
    if cv2.waitKey(1) == ord('q'):
        break

capture.release()
cv2.destroyAllWindows()

In [6]:
import numpy as np
import cv2
import math
#상체 탐지(영상)
conda_path = 'D:\\javaForever\\util\\opencv\\sources\\data\\haarcascades\\'
upper_cascade = cv2.CascadeClassifier(conda_path + 'haarcascade_upperbody.xml')

cap = cv2.VideoCapture('HUN.mp4')
scaling_factor = 0.5

width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)

while True:
    ret, frame1 = cap.read()
    
    if ret:
        frame1 = cv2.resize(frame1, None, fx=scaling_factor, fy=scaling_factor, interpolation = cv2.INTER_AREA)
        gray = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)

        upper_rects = upper_cascade.detectMultiScale(gray, 1.1, 1)
        
        copy_x=0
        copy_y=0
        copy_w=0
        copy_h=0

        for(x, y, w, h) in upper_rects:
            cv2.rectangle(frame1, (x, y), (x+w, y+h), (0, 255, 0), 0)
            copy_x = x
            copy_y = y
            copy_w = w
            copy_h = h
        
            frame2 = frame1[copy_y : copy_y+copy_h, copy_x : copy_x+copy_w]

            frame2 = cv2.resize(frame2, (2*frame2.shape[0], 2*frame2.shape[1]), interpolation = cv2.INTER_CUBIC) #2배로 만들기

            frame1[100:100+frame2.shape[0], 100:100+frame2.shape[1]] = frame2[:frame2.shape[0], :frame2.shape[1]]
            
        cv2.imshow('Face Detector', frame1)

        if cv2.waitKey(1) == 27:
            break
    else:
        break
        
cap.release()
cv2.destroyAllWindows()

ValueError: could not broadcast input array from shape (320,262,3) into shape (140,262,3)

In [1]:
# 어깨 인식
# 오류 잡음
import cv2
import numpy as np
import math

def Rotate(src, degrees):
    if degrees == 90:
        dst = cv2.transpose(src) # 행렬 변경 
        dst = cv2.flip(dst, 1)   # 뒤집기

    elif degrees == 180:
        dst = cv2.flip(src, 0)   # 뒤집기

    elif degrees == 270:
        dst = cv2.transpose(src) # 행렬 변경
        dst = cv2.flip(dst, 0)   # 뒤집기
    else:
        dst = null
    return dst

cap = cv2.VideoCapture('HUN.mp4')

while(cap.isOpened()):
    ret, img = cap.read()
    if ret :
        img = Rotate(img,90)
        cv2.rectangle(img,(300,300),(100,100),(0,255,0),0)
        crop_img = img[300:500, 150:360]
        grey = cv2.cvtColor(crop_img, cv2.COLOR_BGR2GRAY)
        value = (35, 35)
        blurred = cv2.GaussianBlur(grey, value, 0)
        _, thresh1 = cv2.threshold(blurred, 127, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
        #cv2.imshow('Thresholded', thresh1)
        _,contours, hierarchy = cv2.findContours(thresh1.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
        max_area = -1

        for i in range(len(contours)):
            cnt=contours[i]
            area = cv2.contourArea(cnt)
            if(area>max_area):
                max_area=area
                ci=i

        cnt=contours[ci]
        x,y,w,h = cv2.boundingRect(cnt)
        cv2.rectangle(crop_img,(x,y),(x+w,y+h),(0,0,255),0)
        hull = cv2.convexHull(cnt)
        drawing = np.zeros(crop_img.shape,np.uint8)
        cv2.drawContours(drawing,[cnt],0,(0,255,0),0)
        cv2.drawContours(drawing,[hull],0,(0,0,255),0)
        hull = cv2.convexHull(cnt,returnPoints = False)
        defects = cv2.convexityDefects(cnt,hull)
        count_defects = 0
        cv2.drawContours(thresh1, contours, -1, (0,255,0), 3)

        for i in range(defects.shape[0]):
            s,e,f,d = defects[i,0]
            start = tuple(cnt[s][0])
            end = tuple(cnt[e][0])
            far = tuple(cnt[f][0])
            a = math.sqrt((end[0] - start[0])**2 + (end[1] - start[1])**2)
            b = math.sqrt((far[0] - start[0])**2 + (far[1] - start[1])**2)
            c = math.sqrt((end[0] - far[0])**2 + (end[1] - far[1])**2)
            angle = math.acos((b**2 + c**2 - a**2)/(2*b*c)) * 57
            if angle <= 90:
                count_defects += 1
                cv2.circle(crop_img,far,1,[0,0,255],-1)
            #dist = cv2.pointPolygonTest(cnt,far,True)
            cv2.line(crop_img,start,end,[0,255,0],2)
            #cv2.circle(crop_img,far,5,[0,0,255],-1)
        if count_defects == 1:
            cv2.putText(img,"cnt_defects : 1", (5,50), cv2.FONT_HERSHEY_SIMPLEX, 2, 2)
        elif count_defects == 2:
            str = "cnt_defects : 2"
            cv2.putText(img, str, (5,50), cv2.FONT_HERSHEY_SIMPLEX, 1, 2)
        elif count_defects == 3:
            cv2.putText(img,"cnt_defects : 4?", (5,50), cv2.FONT_HERSHEY_SIMPLEX, 2, 2)
        elif count_defects == 4:
            cv2.putText(img,"cnt_defects : 4", (5,50), cv2.FONT_HERSHEY_SIMPLEX, 2, 2)
        else:
            cv2.putText(img,"cnt_defects : ?", (5,50), cv2.FONT_HERSHEY_SIMPLEX, 2, 2)
        #cv2.imshow('drawing', drawing)
        #cv2.imshow('end', crop_img)
        cv2.imshow('Gesture', img)
        all_img = np.hstack((drawing, crop_img))
        cv2.imshow('Contours', all_img)
        k = cv2.waitKey(1)
        if k == 27:
            break
    else :
        break
        
cap.release()
cv2.destroyAllWindows()